In [2]:
pip install langchain-openai

  Using cached langchain_openai-1.0.2-py3-none-any.whl.metadata (1.8 kB)
  Using cached tiktoken-0.12.0-cp312-cp312-win_amd64.whl.metadata (6.9 kB)
  Using cached jsonpatch-1.33-py2.py3-none-any.whl.metadata (3.0 kB)
     ---------------------------------------- 0.0/89.9 kB ? eta -:--:--
     ------------------------------- -------- 71.7/89.9 kB 2.0 MB/s eta 0:00:01
     ---------------------------------------- 89.9/89.9 kB 1.3 MB/s eta 0:00:00
  Using cached tenacity-9.1.2-py3-none-any.whl.metadata (1.2 kB)
  Using cached typing_extensions-4.15.0-py3-none-any.whl.metadata (3.3 kB)
  Using cached distro-1.9.0-py3-none-any.whl.metadata (6.8 kB)
  Using cached tqdm-4.67.1-py3-none-any.whl.metadata (57 kB)
     ---------------------------------------- 0.0/41.5 kB ? eta -:--:--
     ---------------------------------------- 41.5/41.5 kB 1.0 MB/s eta 0:00:00
  Using cached orjson-3.11.4-cp312-cp312-win_amd64.whl.metadata (42 kB)
  Using cached requests_toolbelt-1.0.0-py2.py3-none-any.whl.met

In [4]:
pip install langchain-chroma

  Using cached langchain_chroma-1.0.0-py3-none-any.whl.metadata (1.9 kB)
  Using cached build-1.3.0-py3-none-any.whl.metadata (5.6 kB)
  Using cached pybase64-1.4.2-cp312-cp312-win_amd64.whl.metadata (9.0 kB)
  Using cached uvicorn-0.38.0-py3-none-any.whl.metadata (6.8 kB)
  Using cached posthog-5.4.0-py3-none-any.whl.metadata (5.7 kB)
  Using cached onnxruntime-1.23.2-cp312-cp312-win_amd64.whl.metadata (5.3 kB)
  Using cached opentelemetry_api-1.38.0-py3-none-any.whl.metadata (1.5 kB)
  Using cached opentelemetry_exporter_otlp_proto_grpc-1.38.0-py3-none-any.whl.metadata (2.4 kB)
  Using cached opentelemetry_sdk-1.38.0-py3-none-any.whl.metadata (1.5 kB)
  Using cached tokenizers-0.22.1-cp39-abi3-win_amd64.whl.metadata (6.9 kB)
  Using cached pypika-0.48.9-py2.py3-none-any.whl
  Using cached importlib_resources-6.5.2-py3-none-any.whl.metadata (3.9 kB)
  Using cached grpcio-1.76.0-cp312-cp312-win_amd64.whl.metadata (3.8 kB)
  Using cached bcrypt-5.0.0-cp39-abi3-win_amd64.whl.metadata (10

In [5]:
pip install langchain-core

In [2]:
import os
from getpass import getpass

os.environ["OPENAI_API_KEY"] = os.getenv("OPENAI_API_KEY") \
    or getpass("Enter your OpenAI API key: ")

In [ ]:
import pandas as pd
from datetime import datetime
from langchain_openai import ChatOpenAI, OpenAIEmbeddings
from langchain_chroma import Chroma
from langchain_core.runnables import RunnableMap, RunnablePassthrough
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate

# Load vector DB
CHROMA_DIR = "chroma_eu_laws"
embedding = OpenAIEmbeddings(model="text-embedding-3-small")
vectorstore = Chroma(persist_directory=CHROMA_DIR, embedding_function=embedding)
retriever = vectorstore.as_retriever(search_kwargs={"k": 4})

# RAG chain (same as in app.py)
prompt = ChatPromptTemplate.from_template("""
You are a legal expert specialized in European and GDPR law.
Use the following retrieved context to answer the user's question precisely.
If uncertain, say you don't know.

Context:
{context}

Question:
{question}
""")

llm = ChatOpenAI(model="gpt-4o-mini", temperature=0)
rag_chain = (
    RunnableMap({
        "context": retriever | (lambda docs: "\n\n".join([d.page_content for d in docs])),
        "question": RunnablePassthrough()
    })
    | prompt
    | llm
    | StrOutputParser()
)

In [7]:
questions = [
    "Can I store customer photos for identification?",
    "What are my obligations if I use AI to select job candidates?",
    "Can I track employee activity with software?",
    "Do I need a Data Protection Officer under GDPR?",
    "Can I transfer customer data to the US?",
    "I am building an AI tool for people that could help to easily check if the project they are doing is compliant with EU norms. What should I be aware of laws regarding what I want to do?"
    "I want to implement a chatbot on my website that collects user data. What legal considerations should I keep in mind?",
    "I am developing an AI-driven hiring platform. What regulations should I consider?",
    "I am developping a marketplace that sells shoes online. Is it legal?",
    "I developped an api that i am selling now to companies selling shoes. Is my project compliant with eu norms ?"
]

df = pd.DataFrame({"question": questions})
df["model_answer"] = ""
df["eval_score"] = ""
df["feedback"] = ""

In [8]:
for i, row in df.iterrows():
    q = row["question"]
    print(f"🧩 Processing: {q}")
    answer = rag_chain.invoke(q)
    df.at[i, "model_answer"] = answer

🧩 Processing: Can I store customer photos for identification?


: 

In [ ]:
judge = ChatOpenAI(model="gpt-4o-mini", temperature=0)

def evaluate_answer(question, answer):
    eval_prompt = f"""
You are an expert in EU law. Evaluate the following model answer.

Question: {question}
Answer: {answer}

Rate the answer on:
- Accuracy (0–5)
- Clarity (0–5)
- Legal relevance (0–5)

Return a short feedback and an average score.
"""
    resp = judge.invoke(eval_prompt).content
    return resp

df["evaluation"] = df.apply(lambda x: evaluate_answer(x["question"], x["model_answer"]), axis=1)
df[["eval_score", "feedback"]] = df["evaluation"].str.extract(r'Average Score:\s*(\d+)\s*Feedback:\s*(.*)', expand=True)

In [ ]:
timestamp = datetime.now().strftime("%Y-%m-%d_%H-%M")
df.to_csv(f"evaluation_results_{timestamp}.csv", index=False)
print("✅ Evaluation saved!")

In [ ]:
# Parse scores if GPT returns them structured
import re

def extract_score(text):
    match = re.search(r"(\d(?:\.\d)?)/?5", text)
    return float(match.group(1)) if match else None

df["avg_score"] = df["evaluation"].apply(extract_score)
print("Average score:", df["avg_score"].mean())